In [4]:
%reload_ext autoreload
%autoreload 2

from IPython.display import display_markdown

In [ ]:
import en_vectors_web_lg
nlp = en_vectors_web_lg.load()

In [ ]:
MAX_WORDS_IN_SENTENCE = 300
LIMIT = -1
dataset_id = 'imdb_{}'.format(LIMIT if LIMIT > 0 else 'all')

In [12]:
from sklearn.model_selection import train_test_split
from shared.data import load_imdb

print("Loading training data...")
X_train, y_train = load_imdb('../data/aclImdb/train', limit=LIMIT)

print("Loading test data...")
# lower text count, because of memory problems when trying to load all
test_limit = LIMIT if 0 <= LIMIT <= 1000 else 1000
X_test, y_test = load_imdb('../data/aclImdb/test', limit=test_limit)

Loading training data...
Loading test data...


In [13]:
from machine_learning.plot_helpers import describe_data

describe_data(X_train, y_train)

display_markdown('### Train count: {}'.format(len(X_train)), raw=True)
display_markdown('### Test count:  {}'.format(len(X_test)), raw=True)

### Data sample

,X,y
0,"b""Dan Katzir has produced a wonderful film tha...",1
1,b'If you want Scream or anything like the big-...,1
2,"b""Outlandish premise that rates low on plausib...",0
3,b'Let\'s face it-- if you rented a STDVD seque...,0
4,b'Bizarre Tobe Hooper exercise regarding an un...,0
5,"b'Well, maybe the PC version of this game was ...",0
6,"b'Watching ""Kroko"" I would have liked to leave...",0
7,"b""First of all, I have to start this comment b...",1
8,"b""Very much a film from the times -- extremely...",0
9,"b'""The Invisible Ray"" is part science fiction ...",1


#### Text stats

count                                                 25000
unique                                                24904
top       b"This show comes up with interesting location...
freq                                                      3
Name: X, dtype: object

#### Words length stats

...

#### Labels counts

1    12500
0    12500
Name: y, dtype: int64

1    0.5
0    0.5
Name: y, dtype: float64

### Train count: 25000

### Test count:  1000

In [7]:
from shared.models import KerasModel
from machine_learning.plot_helpers import plot_training

display_markdown('#### Training or loading model', raw=True)

model = KerasModel(
    nlp, dataset_id, max_words_in_sentence=MAX_WORDS_IN_SENTENCE, 
    epochs=50)

try:
    print("Loading model...")
    model.load()
    print("Model '{}' loaded".format(model.filename))
except IOError:
    print("Unable to load model, training...")
    history = model.train(X_train, y_train)
    model.save()
    print("Model '{}' saved".format(model.filename))
    plot_training(history)

Using TensorFlow backend.


#### Training or loading model

Loading model...
Unable to load model, training...
Epoch 1/50
196/196 [==============================] - 22s 112ms/step - loss: 0.3000 - acc: 0.7361
Epoch 2/50
196/196 [==============================] - 20s 103ms/step - loss: 0.2125 - acc: 0.84251s - loss: 0.2132 - a
Epoch 3/50
196/196 [==============================] - 22s 112ms/step - loss: 0.1832 - acc: 0.8663
Epoch 4/50
196/196 [==============================] - 22s 111ms/step - loss: 0.1615 - acc: 0.8789
Epoch 5/50
196/196 [==============================] - 22s 112ms/step - loss: 0.1445 - acc: 0.8919
Epoch 6/50
196/196 [==============================] - 22s 111ms/step - loss: 0.1319 - acc: 0.8986
Epoch 7/50
196/196 [==============================] - 22s 111ms/step - loss: 0.1241 - acc: 0.9027
Epoch 8/50
196/196 [==============================] - 22s 111ms/step - loss: 0.1163 - acc: 0.9079
Epoch 9/50
196/196 [==============================] - 22s 112ms/step - loss: 0.1106 - acc: 0.9122
Epoch 10/50
196/196 [=========================

In [49]:
display_markdown("#### Overview of created convolutional network architecture", raw=True)

display(model.summary())

#### Overview of created convolutional network architecture

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 296, 32)           48032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 32)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 59, 32)            128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 55, 64)            10304     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
__________

None

In [43]:
import numpy as np

from sklearn import pipeline
from shared import transformers

raw_samples = np.array([
    "This movie is, in my opinion, very worth watching!",
    "I kinda liked that movie. Maybe it's not as good as other, but still watchable",
    "I have to warn everybody, this movie is really bad. Actors don't know how to play. It su!*!",
], dtype='object')

display_markdown("#### Raw text", raw=True)

for text in raw_samples:
    display(text)

display_markdown("#### Cleaned text", raw=True)

clear_pipeline = pipeline.Pipeline([
    ('clear', transformers.ClearTextTransformer()),
])

for text in clear_pipeline.transform(raw_samples):
    display(text)

display_markdown("#### Predicted scores", raw=True)
with pd.option_context("display.max_colwidth", -1):
    display(pd.DataFrame({
        "text": pd.Series(raw_samples),
        "score": pd.Series(model.predict_proba(raw_samples)[:, 1].reshape(len(raw_samples))),
    }))


#### Raw text

'This movie is, in my opinion, very worth watching!'

"I kinda liked that movie. Maybe it's not as good as other, but still watchable"

"I have to warn everybody, this movie is really bad. Actors don't know how to play. It su!*!"

#### Cleaned text

'this movie is in my opinion very worth watching!'

"i kinda liked that movie maybe it's not as good as other but still watchable"

"i have to warn everybody this movie is really bad actors don't know how to play it su! !"

#### Predicted scores

,score,text
0,0.957330,"This movie is, in my opinion, very worth watching!"
1,0.718341,"I kinda liked that movie. Maybe it's not as good as other, but still watchable"
2,0.000038,"I have to warn everybody, this movie is really bad. Actors don't know how to play. It su!*!"


In [ ]:
from machine_learning.evaluation import evaluate_and_report
evaluate_and_report(model, X_test, y_test)